In [ ]:
import pandas as pd
import numpy as np
import plotly as plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import random

pop = pd.read_excel(r'population.xlsx')
gdp = pd.read_excel(r'percapitagdp.xlsx')
latlong = pd.read_csv(r'latlong.csv')
df = pop 
df.set_index('Country', inplace=True)
gdp.set_index('Country', inplace=True)
latlong.set_index('Country', inplace=True)

init_notebook_mode(connected=True)

In [ ]:
from math import radians, degrees, sin, cos, asin, acos, sqrt

def great_circle(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    return 6371 * (
        acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2)))

def normalize(ds):
    total = float(ds.sum())
    new = ds / total
    return new

for country in list(gdp.index):
    if country in list(df.index):
        df.at[country, 'Per Capita GDP'] = gdp.at[country, 'PerCapitaGDP']

df['Normalized GDP'] = normalize(df['Per Capita GDP'])

for country in list(latlong.index):
    if country in list(df.index):
        df.at[country, 'Latitude'] = latlong.at[country, 'Latitude']
        df.at[country, 'Longitude'] = latlong.at[country, 'Longitude']

for country in list(df.index):
    if np.isnan(float(df.at[country, "Population"])):
        df = df.drop([country])

for country in list(df.index):
    if np.isnan(df.at[country, "Per Capita GDP"]):
        df = df.drop([country])

for country in list(df.index):
    if np.isnan(df.at[country, "Longitude"]):
        df = df.drop([country])
        
        
for country in list(df.index):
    for country_target in list(df.index):
        if country != country_target:
            df.at[country, 'Distance to ' + country_target] = great_circle(df.at[country, 'Longitude'], 
                                                                       df.at[country, 'Latitude'],
                                                                       df.at[country_target, 'Longitude'],
                                                                       df.at[country_target, 'Latitude'])
df.rename(columns={'Population':'2020'}, inplace=True)

In [ ]:
def weight(country1, country2, year):
    pop_1 = df.at[country1, str(year)] 
    pop_2 = df.at[country2, str(year)]
    gdp_1 = df.at[country1, 'Per Capita GDP']
    gdp_2 = df.at[country2, 'Per Capita GDP'] 
    distance = df.at[country1, 'Distance to ' + country2]
    num =  (-0.00021 * (pop_1 - pop_2)) + (-50 * (gdp_1 - gdp_2) + (-10 * distance))
    return num

def simple_diffusion():
    for i in range(2021, 2031):
        for country in list(df.index):
            df.at[country, str(i)] = (df.at[country, str(i-1)]
                                     + (-0.5 * (df.at[country, str(i-1)] 
                                     - df[str(i)].mean())))

def complex_diffusion():
    prop_df = pd.DataFrame(list(df.index),columns=['Country'])
    prop_df.set_index('Country', inplace=True)
    prop_df['Migrants'] = 0
    prop_df['Immigrants'] = 0
    prop_df['Emigrants'] = 0

    for i in range(2021, 2031):
        
        prop_df['Immigrants'] = 0
        prop_df['Emigrants'] = 0
       
        for country in list(df.index):
            pop = df.at[country, str(i-1)]
            if pop > 0:
                migrability = ((1 / df.at[country, 'Normalized GDP'])  / 11000) + (((random.random() * (2) - 1)) / 550)
                for country_target in list(df.index):
                    if country_target != country:
                        prop_df.at[country_target, 'Migrants'] = weight(country, country_target, (i-1))
                    if country_target == country:
                        prop_df.at[country_target, 'Migrants'] = 0
                prop_df['Migrants'] = normalize(prop_df['Migrants'])
                prop_df['Migrants'] = ((migrability) * pop) * (prop_df['Migrants'])
                prop_df['Immigrants'] += prop_df['Migrants']
                prop_df.at[country, 'Emigrants'] = (migrability) * pop
            else:
                df[str(i)] = 0
        
        df[str(i)] = df[str(i-1)] + prop_df['Immigrants'] - prop_df['Emigrants']
                

df
complex_diffusion()

In [ ]:
for column in df.columns:
    try:
        int(column)
    except:
        df.drop(column, axis=1, inplace=True)

df['Country'] = df.index

col = list(df.columns)
col.remove('Country')

df = pd.melt(df, id_vars=['Country'], value_vars=col,
        var_name='Year', value_name='Population')

In [ ]:
def single_year(year):
    df['Year'] = df['Year'].astype(int)

    df_sected_2020 = df[(df['Year']== year)]

    data = dict (
        type = 'choropleth',
        locations = df_sected_2020['Country'],
        locationmode='country names',
        z=np.log10(df_sected_2020['Population']),
        text = df_sected_2020['Population'],
        colorbar=dict(title='Population',
                      tickvals = [3, 4, 5, 6, 7, 8, 9],
                      ticktext = ['1 thousand', '10 thousand', '100 thousand', '1 million', '10 million','100 million', '1 billion']))

    fig = go.Figure(data=[data])
    plotly.offline.iplot(fig)

def mult_year():
    year = 2020

    df['Year'] = df['Year'].astype(int)

    data_slider = []

    for year in df['Year'].unique():
        df_segmented =  df[(df['Year']== year)]

        data_each_yr = dict(
                            type='choropleth',
                            locations = df_segmented['Country'],
                            z=np.log10(df_segmented['Population']),
                            zmax=10,
                            zmin=3,
                            text = df_segmented['Population'],
                            locationmode='country names',
                            colorbar=dict(title='Population',
                                          tickvals = [3, 4, 5, 6, 7, 8, 9],
                                          ticktext = ['1 thousand', '10 thousand', '100 thousand', '1 million', '10 million','100 million', '1 billion']))

        data_slider.append(data_each_yr)

    steps = []
    for i in range(len(data_slider)):
        step = dict(method='restyle',
                    args=['visible', [False] * len(data_slider)],
                    label='Year {}'.format(i + 2020))
        step['args'][1][i] = True
        steps.append(step)

    sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

    layout = dict(geo=dict(scope='world',
                           projection={'type': 'mercator'}),
                  sliders=sliders)

    fig = dict(data=data_slider,layout=layout)

    plotly.offline.iplot(fig)

mult_year()
